<a href="https://colab.research.google.com/github/Suf1an65/Final-Year-Project/blob/main/notebooks/Final_Scraping_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# INSTALLATION & NODE.JS FIX
# -----------------------------
!apt-get update -qq
!apt-get install -y ffmpeg

# 1. Install Node.js
!apt-get install -y nodejs



# 3. Optional: Verify Node.js is now available to the system
!node --version
# -----------------------------

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  javascript-common libc-ares2 libjs-highlight.js libnode72 nodejs-doc
Suggested packages:
  apache2 | lighttpd | httpd npm
The following NEW packages will be installed:
  javascript-common libc-ares2 libjs-highlight.js libnode72 nodejs nodejs-doc
0 upgraded, 6 newly installed, 0 to remove and 42 not upgraded.
Need to get 13.7 MB of archives.
After this operation, 54.0 MB of additional disk space will be used.
Get:1 http://archive.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.8 MB/s eta 0:00:00


In [ ]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!apt-get install -y nodejs npm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nodejs is already the newest version (12.22.9~dfsg-1ubuntu3.6).
The following additional packages will be installed:
  gyp libjs-events libjs-inherits libjs-is-typedarray libjs-psl
  libjs-source-map libjs-sprintf-js libjs-typedarray-to-buffer libnode-dev
  libnotify-bin libnotify4 libuv1-dev node-abab node-abbrev node-agent-base
  node-ansi-regex node-ansi-styles node-ansistyles node-aproba node-archy
  node-are-we-there-yet node-argparse node-arrify node-asap node-asynckit
  node-balanced-match node-brace-expansion node-builtins node-cacache
  node-chalk node-chownr node-clean-yaml-object node-cli-table node-clone
  node-color-convert node-color-name node-colors node-columnify
  node-combined-stream node-commander node-console-control-strings
  node-copy-concurrently node-core-util-is node-coveralls node-cssom
  node-cssstyle node-debug node-decompress-response node-defaults
  node-delaye

### Loading the data frame (Obselete, will only work with csv files from now on)

## I no longer need to load the excel file as it will be a lot more productive for me to work with the csv file as my dataframe

In [ ]:
'''


# -----------------------------
# LOAD DATAFRAME
# -----------------------------
# Force time columns to be read as strings to avoid Excel datetime conversion
df = pd.read_excel(
    excel_path,
    skiprows=2,
    dtype={
        'Start time in the episode': str,
        'End time in the episode': str
    }
)

# Drop extra columns (keep only first 8)
columns_to_drop = [num for num in range(8, len(df.columns))]
df.drop(df.columns[columns_to_drop], axis=1, inplace=True)

# Add tracking columns
if 'video_path' not in df.columns:
    df['full_video_path'] = ""
if 'status' not in df.columns:
    df['status'] = ""

print("=== DATASET INFO ===")
print(f"Total clips in dataset: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
'''

=== DATASET INFO ===
Total clips in dataset: 1678
Columns: ['Video URL', 'File name of the video clip ', 'Label "truth" or "deception"', 'Start time in the episode', 'End time in the episode', 'Name of the game show', 'Participants name', 'Participants gender', 'full_video_path', 'status']


## I'm going to break this down into blocks
### First block is import statements and packages

In [3]:
import pandas as pd
import numpy as np
import yt_dlp
import os
import glob
import time

### Setting up the paths for directories

In [4]:
# -----------------------------
# PATHS
# -----------------------------
base_dir = '/content/drive/MyDrive/Final-Year-Project'
video_directory = os.path.join(base_dir, 'data/WebScrapedVideos/')
excel_path = os.path.join(base_dir, 'data/spreadsheets/Dolos.xlsx')
cookie_path = os.path.join(base_dir, 'Notebooks/cookies.txt')
output_csv_path_og = os.path.join(base_dir, 'data/spreadsheets/my_dataset_with_paths_improved.csv')
output_csv_path = os.path.join(base_dir, 'data/spreadsheets/my_dataset_with_paths_improved_updated.csv')
urls_dict_path = os.path.join(base_dir, 'data/urls_dict.json')

os.makedirs(video_directory, exist_ok=True)

In [5]:
df_csv = pd.read_csv(output_csv_path_og)
df_csv.head()
df_csv_null_ts = df_csv[df_csv['Start time in the episode'].isnull() | df_csv['End time in the episode'].isnull()]
df_csv_null_ts.count()
#df_csv.head(20)

,0
Video URL,0
File name of the video clip,0
"Label ""truth"" or ""deception""",0
Start time in the episode,0
End time in the episode,0
Name of the game show,0
Participants name,0
Participants gender,0
full_video_path,0
status,0


In [6]:
df = pd.read_csv(output_csv_path)

failed_rows = df[df['status'] == 'Failed']
failed_urls = failed_rows['Video URL'].unique()

print(f"Total unique failed URLs: {len(failed_urls)}\n")
for url in failed_urls:
    print(url)

Total unique failed URLs: 8

https://www.youtube.com/watch?v=X9LG_kwG9uQ&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=30
https://www.youtube.com/watch?v=HpM8HbO6KVQ&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=39
https://www.youtube.com/watch?v=7aahLFGgU4s&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=40
https://www.youtube.com/watch?v=TnL8Ulfl9aM&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=8
https://www.youtube.com/watch?v=JOqMqNlhLOI&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=67
https://www.youtube.com/watch?v=HPlXvaGFJ3I&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=68
https://www.youtube.com/watch?v=YOw1CSQ1Kro&list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b&index=69
https://youtu.be/r_EYquoGsK0?list=PL5feTlxqLhgQd8vMlwhv8P954SXrHpR7b


### TimeStamp functions

In [ ]:

import re # We'll use regex to cleanly extract the numbers


def fix_timestamp(t):
  # Ideal timestamp: hh:mm:ss
  # Hours should always be 0
    """Clean up timestamp strings"""
    if pd.isna(t):
        return None
    #Clean up formats when it has 1 day, then time

    t_str = str(t).strip()

    if "day" in t_str:
        separations = t_str.split(',')

        problem = separations[0].split()
        padding = int(problem[0])
        time_stamps = separations[1].split(":")

        minutes = int(time_stamps[0])
        seconds = int(time_stamps[1])
        hours = 0

        minutes = (padding * 24) + minutes
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

    elif "." in t_str:
        separations = t_str.split('.')

        minutes = int(separations[0])
        seconds = int(separations[1])
        hours = 0
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    else:
      separations = t_str.split(':')

      if len(separations) == 3:
        hours = int(separations[0])
        minutes = int(separations[1])
        seconds = int(separations[2])

        if hours > 0:
          minutes = int(separations[0])
          seconds = int(separations[1])
          hours = 0
          return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
      elif len(separations) == 2:
        minutes = int(separations[0])
        seconds = int(separations[1])
        hours = 0
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
      else:
        print(t_str)
        return t_str

def timestamp_to_seconds(timestamp):
    """Convert HH:MM:SS timestamp to total seconds"""
    if not timestamp:
        return None
    # Assuming fix_timestamp has standardized the string to H:M:S format
    parts = timestamp.split(':')
    hours = int(parts[0])
    minutes = int(parts[1])
    seconds = int(parts[2])
    return hours * 3600 + minutes * 60 + seconds

df['Start time in the episode'] = df['Start time in the episode'].apply(fix_timestamp)
df['End time in the episode'] = df['End time in the episode'].apply(fix_timestamp)


In [ ]:
df = pd.read_csv(output_csv_path_og)
df.to_csv(output_csv_path, index=False)

Main Code for downloading and showing progress

In [ ]:
import json
df = pd.read_csv(output_csv_path)
# -----------------------------
# BATCH CONFIGURATION
# -----------------------------
BATCH_SIZE = 3   # Process 50 at a time
START_INDEX = 1675

# -----------------------------
# CALCULATE BATCH RANGE
# -----------------------------
start_idx = START_INDEX
end_idx = min(START_INDEX + BATCH_SIZE, len(df))


print(f"\n=== BATCH PROCESSING ===")
print(f"Processing rows: {start_idx} to {end_idx-1}")
print(f"Batch size: {end_idx - start_idx}")

# Check how many already done
already_done = df.iloc[start_idx:end_idx]['status'].eq("Success").sum()
if already_done > 0:
    print(f"⚠️  {already_done} clips already downloaded in this batch")
    print(f"Will skip these and process remaining {end_idx - start_idx - already_done}\n")

# -----------------------------
# DOWNLOAD LOOP
# -----------------------------
failure_count = 0
success_count = 0
start_time = time.time()

with open(urls_dict_path, 'r') as f:
    urls_dict = json.load(f)

for index, row in df.iloc[start_idx:end_idx].iterrows():
    # Skip if already successful
    if df.at[index, 'status'] == "Success":
        success_count += 1
        print(f"Row {index}: Already downloaded ✓")
        continue

    video_url = row['Video URL']






    if video_url in urls_dict:
      if urls_dict[video_url] == "FAILED":  # ADD THIS CHECK
        df.at[index, 'status'] = "Failed"
        print(f"Row {index}: Previously failed ✗")
      else:
        df.at[index, 'full_video_path'] = urls_dict[video_url]
        df.at[index, 'status'] = "Success"
        print(f"Row {index}: Already downloaded ✓")
      df.to_csv(output_csv_path, index=False)
      continue

    else:

      ydl_opts = {
      'format': 'bestvideo[height<=720]+bestaudio/best[height<=720]',
      'merge_output_format': 'mp4',
      'remux_video': 'mp4',
      'concurrent_fragment_downloads': 1,
      'cookiefile': cookie_path,
      'nopart': True,
      'noplaylist': True,
      'embedsubs': True,
      'embedmetadata': True,
      'embedchapters': True,
      'outtmpl': os.path.join(video_directory, f'{index:04d}.%(ext)s'),

      }

      with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          try:
            ydl.download([video_url])
            success_count += 1

            full_output_path = os.path.join(video_directory, f'{index:04d}.mp4')

            df.at[index, 'full_video_path'] = full_output_path
            df.at[index, 'status'] = "Success"

            urls_dict[video_url] = full_output_path

            with open(urls_dict_path, 'w') as f:
                json.dump(urls_dict, f, indent=2)

            df.to_csv(output_csv_path, index=False)

          except Exception as e:
            failure_count += 1

            df.at[index, 'status'] = "Failed"
            urls_dict[video_url] = "FAILED"

            with open(urls_dict_path, 'w') as f:
              json.dump(urls_dict, f, indent=2)

            df.to_csv(output_csv_path, index=False)

            print(f"Row {index}: Download failed ✗")

    clip_num = success_count + failure_count + 1 - already_done
    total_to_process = end_idx - start_idx - already_done

    print(f"\n=== Row {index} ({clip_num}/{total_to_process}) ===")
    print(f"Video URL: {video_url[:60]}...")


# -----------------------------
# BATCH SUMMARY
# -----------------------------
elapsed_time = time.time() - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"\n{'='*50}")
print(f"BATCH {START_INDEX}-{end_idx-1} COMPLETE")
print(f"{'='*50}")
print(f"✓ Success: {success_count}")
print(f"✗ Failed: {failure_count}")
print(f"⏱️  Time: {minutes}m {seconds}s")

total_processed_in_run = success_count + failure_count

if total_processed_in_run > 0:
    success_rate = success_count / total_processed_in_run * 100
    print(f"📊 Success rate (current run): {success_rate:.1f}%")

    # Estimate remaining time
    # Changed the condition check to use total_processed_in_run for accuracy
    if total_processed_in_run > 0:
        avg_time_per_clip = elapsed_time / total_processed_in_run
        remaining_clips = len(df) - end_idx
        remaining_batches = (remaining_clips + BATCH_SIZE - 1) // BATCH_SIZE
        estimated_minutes = (remaining_clips * avg_time_per_clip) / 60
        print(f"⏳ Estimated time for remaining {remaining_clips} clips: ~{estimated_minutes:.0f}m ({remaining_batches} batches)")

# Show dataset progress
total_success = df['status'].eq("Success").sum()
total_failed = (~df['status'].eq("Success") & ~df['status'].eq("")).sum()
print(f"\n📁 OVERALL DATASET PROGRESS")
print(f"Total downloaded: {total_success}/{len(df)} ({total_success/len(df)*100:.1f}%)")
print(f"Total failed: {total_failed}")

# Show next steps
if end_idx < len(df):
    print(f"\n🔄 TO CONTINUE:")
    print(f"Change START_INDEX to {end_idx}")
    print(f"Then run this cell again")
else:
    print(f"\n🎉 ALL CLIPS PROCESSED!")
    print(f"Check your CSV: {output_csv_path}")


=== BATCH PROCESSING ===
Processing rows: 1675 to 1677
Batch size: 3
Row 1675: Already downloaded ✓
Row 1676: Already downloaded ✓
Row 1677: Already downloaded ✓

BATCH 1675-1677 COMPLETE
✓ Success: 0
✗ Failed: 0
⏱️  Time: 0m 0s

📁 OVERALL DATASET PROGRESS
Total downloaded: 1480/1678 (88.2%)
Total failed: 198

🎉 ALL CLIPS PROCESSED!
Check your CSV: /content/drive/MyDrive/Final-Year-Project/data/spreadsheets/my_dataset_with_paths_improved_updated.csv
